In [1]:
include("ExperimentDataDictionary.jl");
using Plots
pwd()

"/Users/nadanai263/OneDrive - University of Edinburgh/Coding/Repos/Research/CFPS-metabolic"

In [2]:
# 1. Get experimental data

# Setup the timescale
TSTART = 0.0
TSTOP = 3.0
Ts = 0.1

# Read experimental data into dictionary (originally, 37 metabolite timeseries)
# Take experimental data and interpolate onto simulation timescale
experimental_data_dictionary = ExperimentDataDictionary(TSTART,TSTOP,Ts);

# dictionary links name of metabolite to a struct
# Experiment() has three elements: 
# model_index, interpolated_measurement_array, raw_measurement_array
dat1 = experimental_data_dictionary["CAT"].interpolated_measurement_array

31-element Array{Float64,1}:
 5.4445733e-5         
 0.0001524480500479445
 0.0009034502224604245
 0.0019804290075395754
 0.0034890270144881516
 0.0052134346         
 0.006171236279999999 
 0.007129037959999999 
 0.00808683964        
 0.00904464132        
 0.010002443          
 0.010336370899999999 
 0.0106702988         
 ⋮                    
 0.013007794100000001 
 0.013341722          
 0.013855271200000002 
 0.0143688204         
 0.0148823696         
 0.015395918799999998 
 0.015909468          
 0.0164230172         
 0.0169365664         
 0.0174501156         
 0.017963664799999998 
 0.018477214          

## Solve model

In [3]:
# 1. Load experimental data + model params into dictionary
include("DataFile.jl")
using Plots

TSTART = 0.0
TSTOP = 3.0
Ts = 0.1

# Create the data_dictionary and initialise it with network stoichiometric matrix, 
# experimental data, and all params to 0 (rates), 1 (initial conds, saturation, order),
# and 0.1 (gain).
data_dictionary = DataFile(TSTART,TSTOP,Ts)

# Import initial conditions
initial_condition_vector = vec(broadcast(abs, float(open(readdlm,"params/initial_condition.dat"))))
data_dictionary["INITIAL_CONDITION_ARRAY"] = initial_condition_vector

# Import best fit rate constants
rate_constant_vector = vec(broadcast(abs, float(open(readdlm,"params/rate_constant.dat"))))
data_dictionary["RATE_CONSTANT_ARRAY"] = rate_constant_vector

# Import best fit saturation constants
saturation_constant_array = broadcast(abs, float(open(readdlm,"params/saturation_constant.dat")))
data_dictionary["SATURATION_CONSTANT_ARRAY"] = saturation_constant_array

# Import best fit control parameters
control_constant_array = broadcast(abs, float(open(readdlm,"params/control_constant.dat")))
data_dictionary["CONTROL_PARAMETER_ARRAY"] = control_constant_array


# 2. Solve mass balance with initial conditions
include("MassBalances.jl")
using Sundials;

# Get required stuff from DataFile struct -
TSIM = (0.0,3.0) # tspan
initial_condition_vector = data_dictionary["INITIAL_CONDITION_ARRAY"];

# Use sundials to solve fluxes. MassBalances returns a vector of gradients
prob = ODEProblem(MassBalances,initial_condition_vector,TSIM,data_dictionary)
X = solve(prob,CVODE_BDF(),abstol=1e-9,reltol=1e-9)


# 3. Plot (Fig. 2 of Horvath 2020 paper)

p1 = plot(X, vars=(100))
p2 = plot(X, vars=(98))
p3 = plot(X, vars=(102))
p4 = plot(X, vars=(104))

p5 = plot(X, vars=(103))
p6 = plot(X, vars=(118))
p7 = plot(X, vars=(105))
p8 = plot()

plot(p1, p2, p3, p4, p5,p6,p7,p8, layout = (2,4), legend = false)

This is a kinetic model containing

* 352 species (enzymes, metabolites, TXTL)
* 204 reaction rate constants
* 204 enzymatic decay constnats
* 548 saturation constants
* 34 allosteric control parameters

which is trained on a dataset of 37 metabolite timeseries measurements over 3h.

To fit the parameters, a constrained MCMC method is used with the following bounds:

* TX/TL/mRNA deg: factor of 2 of reference 
* reaction rates: 1 order of magnitude of reference, determined as follows

-enzyme concentrations set to 170 nM; where available, kcat values multiplied with this gives the Vmax. 

Where not available, a geometric mean of Vmax=110mM/h was used, and rates bounded by 2 orders of magnitude.

* enzyme decay: 0 and 1/h == infty and 42 min
* saturation constant: 100 nM and 10 mM
* control gain: 0.05 and 10
* order params: 0.02 and 10

Cost function is sum squared errors.

Procedure: random walk, accept all with lower cost, accept higher cost sets with exponential probability.

*Implementation complete 2020-03-18*

## Project outline

**Aims**

1. Visualise network

2. Visualise rates, concentrations, and fluxes in intuitive way

2. Determine effect of variation of concentration of every enzyme (+/- 1 order of magnitude) on CAT production yield and lifetime; determine optimal enzymatic perturbations on system

3. Couple protein synthesis to enzymatic process